# 例題：XOR

PytorchにおけるNNの学習/推論を行うにあたり、XORの例題を使用します。  
XORとは、$x_1$と$x_2$の片方が1で残りが0であればラベル１、それら以外のケースであればラベル0を入力するような問題で、単純な線形関数ではうまく推論できないです。

この例題を解くために、Pytorchのモジュールを使用していきます。

In [2]:
import os
import numpy as np 
import time

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

### モデル構築の流れ

基本的には以下の流れで実行します。

1. モデルの定義
2. DataLoaderの準備
3. lossやoptimizerなどの準備  
~ここから最適化開始~
4. DataLoaderからバッチの取得
5. バッチをモデルへ入力し予測値を得る
6. 予測値と実測値からLossを計算する
7. ロスに関して各パラメータの勾配を計算する
8. 勾配方向へモデルのパラメータを更新する
9. 3~7の操作をイタレーションの回数行う

#### 1. モデルの定義

XORの例題から、入力は$x_1$と$x_2$の２変数で、ラベルは$0$と$1$のみです。  
また、モデルは活性化関数がtanh, 隠れ層を１つだけ持つネットワークを考えます。

In [19]:
import torch
import torch.nn as nn

Pytorchのモデルは以下のような構成である必要があります。

In [10]:
class TemplateNet(nn.Module):
    
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        # xを入力とした時の計算
        pass

In [9]:
class SimpleClassifier(nn.Module):

    def __init__(self, num_in, num_hid, num_out):
        super().__init__()
        self.layer1 = nn.Linear(num_in, num_hid)
        self.act_fn = nn.Tanh()
        self.layer2 = nn.Linear(num_hid, num_out)
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.act_fn(x)
        x = self.layer2(x)

        return x

In [11]:
model = SimpleClassifier(num_in=2, num_hid=4, num_out=1)
print(model)

SimpleClassifier(
  (layer1): Linear(in_features=2, out_features=4, bias=True)
  (act_fn): Tanh()
  (layer2): Linear(in_features=4, out_features=1, bias=True)
)


この時、モデルが持つパラメータを見てみます。``Tanh``はパラメータを持たないので、layerのパラメータのみ表示されます。

In [13]:
for name, param in model.named_parameters():
    print(f"Parameter {name}, shape {param.shape}")

Parameter layer1.weight, shape torch.Size([4, 2])
Parameter layer1.bias, shape torch.Size([4])
Parameter layer2.weight, shape torch.Size([1, 4])
Parameter layer2.bias, shape torch.Size([1])


注意点としては、layerの定義の仕方です。``self.?``のような形で定義しないと以下のように認識/登録されないです。

In [16]:
class SimpleClassifier(nn.Module):

    def __init__(self, num_in, num_hid, num_out):
        super().__init__()
        layer1 = nn.Linear(num_in, num_hid)
        act_fn = nn.Tanh()
        layer2 = nn.Linear(num_hid, num_out)
        self.list_layer = [layer1, act_fn, layer2]
    
    def forward(self, x):
        for layer in self.list_layer:
            x = layer(x)

        return x

model = SimpleClassifier(num_in=2, num_hid=4, num_out=1)
print(model)

for name, param in model.named_parameters():
    print(f"Parameter {name}, shape {param.shape}")

SimpleClassifier()


上記のように定義したい場合は、``nn.ModuleList``や``nn.ModuleDict``, ``nn.Sequential``を使用します。ここでは、``nn.ModuleList``の例のみを見ます。

In [17]:
class SimpleClassifier(nn.Module):

    def __init__(self, num_in, num_hid, num_out):
        super().__init__()
        layer1 = nn.Linear(num_in, num_hid)
        act_fn = nn.Tanh()
        layer2 = nn.Linear(num_hid, num_out)
        self.list_layer = nn.ModuleList([layer1, act_fn, layer2])
    
    def forward(self, x):
        for layer in self.list_layer:
            x = layer(x)

        return x

model = SimpleClassifier(num_in=2, num_hid=4, num_out=1)
print(model)

for name, param in model.named_parameters():
    print(f"Parameter {name}, shape {param.shape}")

SimpleClassifier(
  (list_layer): ModuleList(
    (0): Linear(in_features=2, out_features=4, bias=True)
    (1): Tanh()
    (2): Linear(in_features=4, out_features=1, bias=True)
  )
)
Parameter list_layer.0.weight, shape torch.Size([4, 2])
Parameter list_layer.0.bias, shape torch.Size([4])
Parameter list_layer.2.weight, shape torch.Size([1, 4])
Parameter list_layer.2.bias, shape torch.Size([1])


#### 2. DataLoaderの準備

pytorchでデータを扱う際は、``Dataset``と``DataLoader``を使用します。  
シンプルには、``Dataset``はi番目のデータを取得するためのクラスで、``DataLoader``はバッチ処理などを効率的に実装できるクラスです。

In [18]:
# dataを効率的に扱うモジュールをインポート
import torch.utils.data as data

``Dataset``クラスはi番目のデータを返す``__getitem__()``とデータのサイズを返す``__len__()``を持ちます。

In [21]:
class XORDataset(data.Dataset):

    def __init__(self, size, std=0.1):
        """
        Inputs:
            size - Number of data points we want to generate
            std - Standard deviation of the noise (see generate_continuous_xor function)
        """
        super().__init__()
        self.size = size
        self.std = std
        self.generate_continuous_xor()

    def generate_continuous_xor(self):
        data = torch.randint(low=0, high=2, size=(self.size, 2), dtype=torch.float32)
        label = (data.sum(dim=1) == 1).to(torch.long)
        data += self.std * torch.randn(data.shape)

        self.data = data
        self.label = label

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        data_point = self.data[idx]
        data_label = self.label[idx]
        return data_point, data_label

In [23]:
dataset = XORDataset(size=200)
print(dataset.size)
print(dataset[0])

200
(tensor([0.9210, 0.9966]), tensor(0))


``DataLoader``クラスは上記で定義した``Dataset``の``__getitem__``を使用して、バッチ処理などをよしなに実行してくれます。

オプションは以下の通りです。
- batch_size: バッチサイズを指定します
- shuffle: データセットの並び順をシャッフルするかどうか
- pin_memory: GPU上のメモリにデータをコピーします。サイズが大きい時には有効ですが、GPUのメモリを消費するので単なる検証や推論の時には必要ないです。
- drop_last: batch_sizeでデータの数を割り切れない時の余りを使用するかどうか（訓練時のみバッチサイズを一定に保つために必要）

In [27]:
data_loader = data.DataLoader(dataset, batch_size=8, shuffle=True, drop_last=True)
print(data_loader)

data_inputs, data_labels = next(iter(data_loader))
print("Data inputs", data_inputs.shape, "\n", data_inputs)
print("Data labels", data_labels.shape, "\n", data_labels)

Data inputs torch.Size([8, 2]) 
 tensor([[ 1.0277,  1.0672],
        [-0.0125,  1.0780],
        [ 0.8900,  0.0438],
        [ 0.0583,  1.0024],
        [-0.0013, -0.1857],
        [-0.1804,  0.1562],
        [ 1.1222,  0.8559],
        [ 0.9104,  0.9939]])
Data labels torch.Size([8]) 
 tensor([0, 1, 1, 1, 0, 0, 0, 0])


#### 最適化

